In [40]:
import numpy as np
import pandas as pd
import warnings,gc,os,time,math
import pymysql
warnings.filterwarnings('ignore')
from dateutil.relativedelta import relativedelta
from datetime import datetime
from tqdm import tqdm_notebook

In [41]:
filename='ud20201019'
num='_34' 
today='20201023'

datap='/home/jupyter/LQ/VW/data/'
new_datap='/home/jupyter/LCG/dazhong/V2/%s/'%(filename) #读取阿桂的数据
savep='/home/jupyter/LQ/VW/20191217_VW_下次保养里程预测/20201023自动化更新数据/%s%s/'%(today,str(num))

if os.path.exists(savep):
    print('True')
else:
    os.mkdir(savep)
    
s1=time.time()

True


In [42]:
def readFromdb94(db,tablename,where_select):
    dbconn=pymysql.connect(
    host="192.168.1.94",database=db,user="root",password="Datauser@2017",port=3306,charset='utf8')
    sqlcmd="select * from "+tablename+where_select
    print(sqlcmd)
    a=pd.read_sql(sqlcmd,dbconn)
    return(a)

def check_df(df):
    df=df.drop_duplicates()
    print(df.shape)
    print(df[['VIN','修理日期']].drop_duplicates().shape)
    print(df['VIN'].nunique())

def check_run_time(s):
    r=(time.time()-s)/60
    return "Run Mins: %f"%r

def get_car(df,old_name):
    df[old_name]=df[old_name].apply(lambda x:str(x).lower())
    pattern='辉昂|Phideon|phideon'
    df.loc[df[old_name].str.contains(pattern),'一级车系']='辉昂'
    df.loc[df[old_name].str.contains('混'),'二级车系']='电动车'
    pattern='途昂|Teramont|teramont'
    df.loc[df[old_name].str.contains(pattern),'一级车系']='途昂'
    df.loc[df[old_name].str.contains('混'),'二级车系']='电动车'
    pattern='帕萨特|Passat|passat|新领驭|领驭'
    df.loc[df[old_name].str.contains(pattern),'一级车系']='帕萨特'
    df.loc[df[old_name].str.contains('混'),'二级车系']='电动车'
    pattern='途观|Tiguan|tiguan'
    df.loc[df[old_name].str.contains(pattern),'一级车系']='途观'
    df.loc[df[old_name].str.contains('混'),'二级车系']='电动车'
    pattern='凌渡|Lamando|lamando'
    df.loc[df[old_name].str.contains(pattern),'一级车系']='凌渡'
    df.loc[df[old_name].str.contains('混'),'二级车系']='电动车'
    pattern='朗行|朗逸|朗境|Lavida|lavida'
    df.loc[df[old_name].str.contains(pattern),'一级车系']='朗逸'
    df.loc[df[old_name].str.contains('混'),'二级车系']='电动车'
    pattern='途安|Touran|touran'
    df.loc[df[old_name].str.contains(pattern),'一级车系']='途安'
    df.loc[df[old_name].str.contains('混'),'二级车系']='电动车'
    pattern='Polo|polo|波罗'
    df.loc[df[old_name].str.contains(pattern),'一级车系']='Polo'
    df.loc[df[old_name].str.contains('混'),'二级车系']='电动车'
    pattern='桑塔纳|Santana|santana'
    df.loc[df[old_name].str.contains(pattern),'一级车系']='桑塔纳'
    df.loc[df[old_name].str.contains('混'),'二级车系']='电动车'
    pattern='途岳|tharu|Tharu'
    df.loc[df[old_name].str.contains(pattern),'一级车系']='途岳'
    df.loc[df[old_name].str.contains('混'),'二级车系']='电动车'
    pattern='途铠|t-cross'
    df.loc[df[old_name].str.contains(pattern),'一级车系']='途铠'
    df.loc[df[old_name].str.contains('混'),'二级车系']='电动车'
    df.loc[df['二级车系'].isnull(),'二级车系']='非电动车'
    return df

def get_car_family(df,new_name):
    high=['辉昂','途昂']
    mid_high=['帕萨特','帕萨特电动','途观','途观电动']
    mid=['凌渡','朗逸','朗逸电动','途安','途岳']
    low=['Polo','桑塔纳','途铠']
    df.loc[df[new_name].isin(high),'车型档次']='高档'
    df.loc[df[new_name].isin(mid_high),'车型档次']='中高档'
    df.loc[df[new_name].isin(mid),'车型档次']='中低档'
    df.loc[df[new_name].isin(low),'车型档次']='低档'
    return df

def get_revise_types(repare_maintain_info):
    repare_maintain_info['修理类型']=repare_maintain_info['修理类型'].fillna('没有进店过')
    pattern = r'常规保养|7500保养|定期保养|快速保养|普修有保养|润滑保养|维修保养|预约保养|A类保养|17保养|常保|快保|换油|更换机油'
    repare_maintain_info.loc[repare_maintain_info['修理类型'].str.contains(pattern), u'常规保养'] = 1
    repare_maintain_info.loc[repare_maintain_info['修理类型'] == '保养', u'常规保养'] = 1
    pattern = r'质量担保保养|首次|首保|新帕.*首'
    repare_maintain_info.loc[repare_maintain_info[u'修理类型'].str.contains(pattern), u'首保'] = 1
    pattern = r'附件安装|附件加装|附件销售'
    repare_maintain_info.loc[repare_maintain_info[u'修理类型'].str.contains(pattern), u'附件安装'] = 1
    pattern = r'事故|中保'
    repare_maintain_info.loc[repare_maintain_info[u'修理类型'].str.contains(pattern), u'事故车维修'] = 1
    pattern = r'一般|普通维修|普通修理|普修|预约修理|小修|延保维修|大客户修理|三包维修|定点修理空'
    repare_maintain_info.loc[repare_maintain_info[u'修理类型'].str.contains(pattern), u'一般维修'] = 1
    pattern = r'索赔|理赔|受控|索'
    repare_maintain_info.loc[repare_maintain_info[u'修理类型'].str.contains(pattern), u'普通索赔'] = 1
    pattern = r'活动|优惠维修|积分兑换|赠|厂家专项|免费检测|无费用'
    repare_maintain_info.loc[repare_maintain_info[u'修理类型'].str.contains(pattern), u'服务活动'] = 1 
    repare_maintain_info.loc[repare_maintain_info[u'修理类型'].str.contains('二手车认证维修'), u'二手车认证维修'] = 1 
    repare_maintain_info.loc[repare_maintain_info[u'修理类型'].str.contains('内部车辆维修|内部修理'),u'内部车辆维修'] = 1 
    return repare_maintain_info

# (1)LSV
# (2)第4位:车身型式
# (3)第5位:变速器
# (4)第6位:乘员保护系统
# (5)第7/8位:车辆等级
# # (6)第9位:检验位
# (7)第10位:生产年份
# (8)第11位:装配厂
def get_user(df):
    df['VIN']=df['VIN'].apply(str)
    df['brand']=df['VIN'].apply(lambda x:str(x)[:3])
    df['车身型式']=df['VIN'].apply(lambda x:x[3])
    df['变速器']=df['VIN'].apply(lambda x:x[4])
    df['乘员保护系统']=df['VIN'].apply(lambda x:x[5])
    df['车辆等级']=df['VIN'].apply(lambda x:x[6:8])
    df['生产年份']=df['VIN'].apply(lambda x:x[9])
    df['装配厂']=df['VIN'].apply(lambda x:x[10])
    for col in ['车身型式', '变速器', '乘员保护系统', '车辆等级', '生产年份', '装配厂']:
        df[col]=df[col].astype(str)
    df['用户属性']=list(map(lambda a,b,c,d,e,f:a+','+b+','+c+','+d+','+e+','+f,df['车身型式'],df['变速器'],
                        df['乘员保护系统'],df['车辆等级'],df['生产年份'],df['装配厂']))
    return df


def get_miss_cols(df):
    tt=pd.isnull(df).sum().reset_index()
    tt.columns=['字段名','缺失值']
    tt['缺失值占比']=tt['缺失值']/df.shape[0]
    tt=tt.sort_values(by=['缺失值占比'],ascending=False).reset_index(drop=True)
    miss_cols=tt[tt['缺失值占比']>0]['字段名'].unique().tolist()
    miss_df=tt[tt['缺失值占比']>0]
    return miss_cols,miss_df

def check_every_col_value(df):
    cols=list(df.columns)
    cols.remove('VIN')
    for col in cols:
        print(col)
        print(df[col].unique().tolist())
        print('-'*15)

In [43]:
def get_veh_base_info(new_datap):
    
    print('############客户标签车辆基础数据############')
    dd1=pd.read_pickle(new_datap+'%s_车辆基础数据%s.pickle'%(filename,num))
    print('原始字段有: ',dd1.columns.tolist())
    
    needs=['VIN', '使用性质', '出厂日期', '发动机-机组号', '变速箱-机组号', '技术代码', '技术代码1',
       '换购', '排量', '档位', '购车时间', '车型描述', '车型代码', '进气方式']
    dd1=dd1[needs].drop_duplicates()
    dd1['购车时间']=pd.to_datetime(dd1['购车时间'])
    dd1=dd1.sort_values(by=['VIN','购车时间'],ascending=False).drop_duplicates(['VIN'],keep='last').reset_index(drop=True)
    
    dd1.loc[dd1['技术代码']=='0','技术代码']=np.nan
    dd1.loc[dd1['排量']=='0','排量']=np.nan
    dd1.loc[~(dd1['进气方式'].isin(['涡轮增压','自然吸气'])),'进气方式']=np.nan
    dd1=dd1.drop_duplicates().reset_index(drop=True)
    
    cols=list(dd1.columns)
    cols.remove('VIN')
    for col in cols:
        tmp=dd1[dd1[col].notnull()][['VIN',col]].drop_duplicates()
        print(col)
        print(tmp.shape)
        print(tmp['VIN'].nunique())
        print('-'*30)
        del dd1[col]
        dd1=dd1.merge(tmp,on=['VIN'],how='left')
    dd1=dd1.drop_duplicates().reset_index(drop=True)

    return dd1

In [44]:
def get_member_info(new_datap):
    
    print('############会员_账户行为############')
    dd2=pd.read_pickle(new_datap+'%s_会员信息%s.pickle'%(filename,num))
    print('原始字段有: ',list(dd2.columns))
        
    if 'vin码' in dd2.columns.tolist():
        dd2.rename(columns={'vin码':'VIN'},inplace=True)
    cols=['VIN','会员卡生成时间', '会员卡类型', '会员卡状态', '性别','是否绑定微信']
    dd2=dd2[cols].drop_duplicates()
    dd2['会员卡生成时间']=pd.to_datetime(dd2['会员卡生成时间'])
    dd2=dd2.sort_values(by=['VIN','会员卡生成时间'],ascending=False).drop_duplicates(['VIN'],keep='last').reset_index(drop=True)
    dd2['会员卡年龄']=(pd.to_datetime(today)-dd2['会员卡生成时间']).apply(lambda x:math.ceil(x.days/180))
    del dd2['会员卡生成时间']
    dd2=dd2.drop_duplicates().reset_index(drop=True)
    gender_dic2={'MALE':'男','FEMALE':'女','UNKNOWN':'未知'}
    dd2['性别1']=dd2['性别'].map(gender_dic2)
    del dd2['性别']
    
    cols=dd2.columns.tolist()
    cols.remove('VIN')
    for col in cols:
        tmp=dd2[dd2[col].notnull()][['VIN',col]].drop_duplicates()
        print(col)
        print(tmp.shape)
        print(tmp['VIN'].nunique())
        print('-'*30)
        del dd2[col]
        dd2=dd2.merge(tmp,on=['VIN'],how='left')
        
    dd2=dd2.drop_duplicates().reset_index(drop=True)
    
    return dd2

In [45]:
def get_cust_info(new_datap):
    print('############客户标签车主基础数据############')
    dd4=pd.read_pickle(new_datap+'%s_车主基础数据%s.pickle'%(filename,num))
    print('原始字段有: ',dd4.columns.tolist())
        
    cols=['VIN','车主性质','性别','车辆类型']
    dd4=dd4[cols].drop_duplicates()

    gender_dic1={'男':'男','女':'女','没有回答':'未知'}
    dd4['性别']=dd4['性别'].map(gender_dic1)
    dd4.loc[dd4['车辆类型']=='nan','车辆类型']=np.nan

    cols=dd4.columns.tolist()
    cols.remove('VIN')
    for col in cols:
        tmp=dd4[dd4[col].notnull()][['VIN',col]].drop_duplicates()
        print(col)
        print(tmp.shape)
        print(tmp['VIN'].nunique())
        print('-'*30)
        del dd4[col]
        dd4=dd4.merge(tmp,on=['VIN'],how='left')

    dd4=dd4.drop_duplicates().reset_index(drop=True)
    
    return dd4

In [46]:
def get_vehicle(new_datap):
    dd1=get_veh_base_info(new_datap)
    dd2=get_member_info(new_datap)
    dd4=get_cust_info(new_datap)
    
    print('############合并,outer merge############')
    veh=dd1.merge(dd2,on=['VIN'],how='outer').merge(dd4,on=['VIN'],how='outer')
    veh.loc[veh['性别'].isnull(),'性别']=veh['性别1']
    del veh['性别1']
    
    veh.loc[veh['技术代码'].isnull(),'技术代码']=veh['技术代码1']
    del veh['技术代码1']
    
    veh=get_user(veh)
    
    veh['出厂日期']=pd.to_datetime(veh['出厂日期']).dt.normalize()  
    veh['购车时间']=pd.to_datetime(veh['购车时间']).dt.normalize()  
    
    print('一级车系和二级车系和车型档次')
    veh=get_car(veh,'车型描述') #一级车系,二级车系
    veh=get_car_family(veh,'一级车系') #车型档次 
    del veh['车型描述']
    
    veh=veh.drop_duplicates().reset_index(drop=True)
    
    return veh

In [47]:
vehicle=get_vehicle(new_datap)
print(vehicle.shape)
print(vehicle['VIN'].nunique())

############客户标签车辆基础数据############
原始字段有:  ['ID', 'NET_CODE', 'VIN', '使用性质', '出厂日期', '发动机-机组号', '变速箱-机组号', '品牌', '技术代码', '技术代码1', '换购', '排量', '档位', '维修车系', '维修车系年款', '购车时间', '购车经销商代码', '车型', '车型_EN', '车型代码', '车型年份', '车型描述', '车型级别', '车型颜色代码', '车牌号', '进气方式']
使用性质
(801479, 2)
801479
------------------------------
出厂日期
(850310, 2)
850310
------------------------------
发动机-机组号
(818548, 2)
818548
------------------------------
变速箱-机组号
(818548, 2)
818548
------------------------------
技术代码
(818038, 2)
818038
------------------------------
技术代码1
(803816, 2)
803816
------------------------------
换购
(850310, 2)
850310
------------------------------
排量
(818038, 2)
818038
------------------------------
档位
(818038, 2)
818038
------------------------------
购车时间
(850286, 2)
850286
------------------------------
车型描述
(850310, 2)
850310
------------------------------
车型代码
(850310, 2)
850310
------------------------------
进气方式
(818038, 2)
818038
------------------------------
############会员_账户行为########

### <font color=red>2.处理主单

In [48]:
def get_one_day_revise_types(df):
    def revise_type_merge(a,b,c,d,e,f,g,h,i):
        lst=[]
        if a==1:
            lst.append('首保')
        if b==1:
            lst.append('常规保养')
        if c==1:
            lst.append('附件安装')
        if d==1:
            lst.append('一般维修')
        if e==1:
            lst.append('事故车维修')
        if f==1:
            lst.append('普通索赔')
        if g==1:
            lst.append('服务活动')
        if h==1:
            lst.append('二手车认证维修')
        if i==1:
            lst.append('内部车辆维修')
        return ','.join(lst)

    df['修理类型_new']=list(map(lambda a,b,c,d,e,f,g,h,i:revise_type_merge(a,b,c,d,e,f,g,h,i),
                                 df['首保'],df['常规保养'],df['附件安装'],df['一般维修'],
                                 df['事故车维修'],df['普通索赔'],df['服务活动'],
                                 df['二手车认证维修'],df['内部车辆维修']))
    return df

In [49]:
df1=pd.read_pickle(new_datap+'%s_维修主单信息%s.pickle'%(filename,num))
print('原始字段有: ',df1.columns.tolist())

# needs=['VIN','公里数','经销商代码','结算单号','修理日期','修理类型',
#        '维修费','折前维修费','工时费','零件费','车辆型号','委托书号']
needs=['VIN','公里数','经销商代码','结算单号','修理日期','修理类型',
       '维修费','折前维修费','工时费','零件费','委托书号']
df1=df1[needs].drop_duplicates().reset_index(drop=True)
df1.loc[df1['折前维修费'].isnull(),'折前维修费']=df1['维修费']
df1['修理日期']=pd.to_datetime(df1['修理日期']).dt.normalize()
df1['公里数']=df1['公里数'].astype(int).astype(float)
df1['修理类型']=df1['修理类型'].apply(lambda x:str(x).replace(';',',').replace(' ',''))
df1.rename(columns={'折前维修费':'维修金额_折前','维修费':'维修金额'},inplace=True)
orders=['VIN','修理日期','公里数','修理类型',
        '经销商代码','结算单号','委托书号','维修金额','维修金额_折前','工时费', '零件费']
df1=df1[orders].drop_duplicates().reset_index(drop=True)

df1=df1.sort_values(by=['VIN','修理日期'],ascending=False).reset_index(drop=True)
for col in ['维修金额','维修金额_折前','工时费', '零件费','公里数']:
    df1[col]=df1[col].astype(float)
df1=get_revise_types(df1)
check_df(df1)

print('1万公里的实际的最后一次保养数据也从这个zhudan_merged_org.csv表单中提取,一定要筛选出来保养数据!!!')
df1.to_pickle(savep+'zhudan_merged_org.pkl')

print("############提取匹配零件表单的数据,与新增的原生数据是不一样的匹配方式############")
zhudan_pipei1=df1[['VIN','修理日期','经销商代码','委托书号','结算单号']].drop_duplicates()
tmp=df1[['VIN','修理日期','公里数']].drop_duplicates().groupby(['VIN','修理日期'])['公里数'].max().reset_index()
zhudan_pipei1=zhudan_pipei1.merge(tmp,on=['VIN','修理日期'],how='left')
print('#仅能匹配15selected中的零件数据,不能匹配新增数据!!!')
zhudan_pipei1.to_pickle(savep+'zhudan_pipei_lingjian_15selected.pkl')

原始字段有:  ['VIN', '交车日期', '交车时间', '修理日期', '修理时间', '修理类型', '公里数', '其他费用', '创建时间', '委托书号', '工时费', '年份', '总金额', '折前其他费用', '折前工时费', '折前总金额', '折前维修费', '折前零件费', '服务顾问', '牌照号', '索赔标记', '经销商代码', '结算单号', '维修费', '负结算标志', '送修人名称', '送修人电话', '零件费']
(5470641, 20)
(5149801, 2)
804533
1万公里的实际的最后一次保养数据也从这个zhudan_merged_org.csv表单中提取,一定要筛选出来保养数据!!!
############提取匹配零件表单的数据,与新增的原生数据是不一样的匹配方式############
#仅能匹配15selected中的零件数据,不能匹配新增数据!!!


In [50]:
def clear_shoubao(zhudan1):
    print("####################清洗首保数据#####################")
    zhudan1=zhudan1.sort_values(by=['VIN','修理日期'],ascending=False).reset_index(drop=True)
    feishoubao=zhudan1[zhudan1['首保']!=1]
    shoubao=zhudan1[zhudan1['首保']==1]
    tmp=shoubao.groupby('VIN')['修理日期'].nunique().reset_index(name='首保次数')
    shoubao=shoubao.merge(tmp,on=['VIN'],how='left')
    print('(1).有%d个VIN出现首保脏数据'%(shoubao[shoubao['首保次数']>=2]['VIN'].nunique()))

    shoubao2=shoubao[shoubao['首保次数']==1]
    shoubao1=shoubao[shoubao['首保次数']>=2]
    shoubao1['时间差']=(shoubao1['修理日期']-shoubao1['购车时间']).apply(lambda x:x.days)
    tmp=shoubao1.groupby('VIN')['时间差'].agg({'max','min'}).reset_index()
    tmp['时间差1']=tmp['min']
    tmp.loc[tmp['时间差1']<0,'时间差1']=tmp[tmp['时间差1']<0]['max']
    tmp=tmp[['VIN','时间差1']].drop_duplicates()
    shoubao1=shoubao1.merge(tmp,on=['VIN'],how='left')
    shoubao1=shoubao1[shoubao1['时间差']==shoubao1['时间差1']].drop_duplicates().reset_index(drop=True)
    print('(2).检查脏首保数据是否正确了:')
    check_df(shoubao1)
    del shoubao1['时间差']
    del shoubao1['时间差1']
    
    print('(3).检查清洗以后的主单数据:')
    zhudan2=pd.concat([shoubao1,shoubao2,feishoubao],axis=0)
    del zhudan2['首保次数']
    check_df(zhudan2)
    
    print('(4).检查首保:')
    check=zhudan2[zhudan2['首保']==1]
    check_df(check)

    del zhudan1
    del tmp
    del shoubao
    del feishoubao
    del shoubao1
    del shoubao2
    del check
    gc.collect()
    
    return zhudan2

In [51]:
def clear_zhudan(df,veh):
    
    def str_merge(lst):
        nlst=[]
        for i in lst:
            ss=str(i).split(',')
            for j in ss:
                if j not in nlst:
                    nlst.append(j)
        return ','.join(nlst)
    
    print('################开始转换成vin+date的宽表################')
    df['修理日期']=pd.to_datetime(df['修理日期']).dt.normalize()
    zhudan=df.groupby(['VIN','修理日期']).agg({'公里数':'max','经销商代码':str_merge,
                                           '修理类型':str_merge,'维修金额':'sum',
                                           '维修金额_折前':'sum','工时费':'sum', '零件费':'sum'}).reset_index()
    zhudan['brand']=zhudan['VIN'].apply(lambda x:str(x)[:3])
    zhudan=get_revise_types(zhudan)
    check_df(zhudan)
    
    print("########加入常去经销商########")
    changqu=df[['VIN','修理日期','经销商代码']].drop_duplicates().groupby(['VIN','经销商代码'])['经销商代码'].count().reset_index(name='次数')
    changqu=changqu.sort_values(by=['VIN','次数'],ascending=False).drop_duplicates(subset=['VIN'],keep='first').drop_duplicates()
    del changqu['次数']
    changqu.rename(columns={'经销商代码':'常去经销商'},inplace=True)
    print(changqu.shape)
    print(changqu['VIN'].nunique())
    zhudan=zhudan.merge(changqu,on=['VIN'],how='left')
    
    print('#############加入第一次进店时间##########')
    first=df.sort_values(by=['VIN','修理日期'],ascending=False).drop_duplicates(['VIN'],keep='last').reset_index(drop=True)
    first=first[['VIN','修理日期']].drop_duplicates().rename(columns={'修理日期':'第一次修理日期'})
#     first['date']=(pd.to_datetime(first['修理日期'])).apply(lambda x:datetime.strftime(x,'%Y-%m')).apply(str)
#     first['date']=first['date'].apply(lambda x:x+'-01').apply(str)
#     first['date']=pd.to_datetime(first['date'])
    changqu=changqu.merge(first,on=['VIN'],how='left')
    
    print('########加入购车时间########')
    veh1=veh[['VIN','购车时间','出厂日期']].drop_duplicates().reset_index(drop=True)
    zhudan1=zhudan.merge(veh1,on=['VIN'],how='left')
    zhudan1['购车时间']=pd.to_datetime(zhudan1['购车时间'])
    zhudan1['出厂日期']=pd.to_datetime(zhudan1['出厂日期'])
    zhudan1['修理日期']=pd.to_datetime(zhudan1['修理日期'])

    wrong_purchase_vin=zhudan1[zhudan1['修理日期']<zhudan1['购车时间']]
    print('有多少vin出现了购车时间>修理日期:',wrong_purchase_vin['VIN'].nunique())
    
    gouche_weixiu=zhudan1[zhudan1['修理日期']==zhudan1['购车时间']]
    print('有多少vin出现了购车那天就被记录了维修的:',gouche_weixiu['VIN'].nunique())
    print('有多少记录出现了购车那天就被记录了维修的:',len(gouche_weixiu))
    
    zhudan1.loc[zhudan1['购车时间'].isnull(),'购车时间']=pd.to_datetime('1990-01-01')
    zhudan1.loc[zhudan1['VIN'].isin(set(wrong_purchase_vin['VIN'])),'购车时间']=pd.to_datetime('1990-01-01')
    
    print('1.先用非空的出厂日期补缺')
    zhudan1.loc[zhudan1['购车时间']==pd.to_datetime('1990-01-01'),'购车时间']=zhudan1[zhudan1['购车时间']==pd.to_datetime('1990-01-01')]['出厂日期']

    print('2.再用大众规则中的生产年份补缺')
    zhudan1['生产年份']=zhudan1['VIN'].apply(lambda x:x[9]).astype(str)

    purchase_dic={'T':'1995-01-01', 'V':'1996-01-01', 'W':'1997-01-01','X':'1998-01-01',
                  'Y':'1999-01-01','0':'2000-01-01','1':'2001-01-01','2':'2002-01-01',
                  '3':'2003-01-01','4':'2004-01-01','5':'2005-01-01','6':'2006-01-01',
                  '7':'2007-01-01','8':'2008-01-01','9':'2009-01-01',
                  'A':'2010-01-01','B':'2011-01-01','C':'2012-01-01','D':'2013-01-01','E':'2014-01-01',
                  'F':'2015-01-01','G':'2016-01-01','H':'2017-01-01','J':'2018-01-01','K':'2019-01-01',
                  'L':'2020-01-01'}
    zhudan1['生产年份1']=zhudan1['生产年份'].map(purchase_dic)
    zhudan1['购车时间']=pd.to_datetime(zhudan1['购车时间'])
    zhudan1['生产年份1']=pd.to_datetime(zhudan1['生产年份1'])
    zhudan1.loc[(zhudan1['购车时间'].isnull())&(zhudan1['brand']=='LSV'),'购车时间']=zhudan1[(zhudan1['购车时间'].isnull())&(zhudan1['brand']=='LSV')]['生产年份1']

    c1=zhudan1[zhudan1['购车时间'].isnull()]
    print('还有多少VIN的购车时间是空置:',c1['VIN'].nunique())
    
    c2=zhudan1[zhudan1['修理日期']<zhudan1['购车时间']]
    print('还有多少VIN的购车时间>修理日期:',c2['VIN'].nunique())
    
    print('3.用修理日期的最早时间补缺')
    first=zhudan1.sort_values(by=['VIN','修理日期'],ascending=False).drop_duplicates(['VIN'],keep='last')
    first=first[['VIN','修理日期']].drop_duplicates()
    first['year']=(pd.to_datetime(first['修理日期'])).apply(lambda x:x.year).apply(str)
    first['date']=first['year'].apply(lambda x:x+'-01-01').apply(str)
    first['date']=pd.to_datetime(first['date'])
    zhudan1=zhudan1.merge(first[['VIN','date']].drop_duplicates(),on='VIN',how='left')
    
    zhudan1.loc[zhudan1['购车时间'].isnull(),'购车时间']=zhudan1[zhudan1['购车时间'].isnull()]['date']
    
    vinlst=zhudan1[zhudan1['修理日期']<zhudan1['购车时间']]['VIN'].unique().tolist()
    zhudan1.loc[zhudan1['VIN'].isin(vinlst),'购车时间']=zhudan1[zhudan1['VIN'].isin(vinlst)]['date']
    
    zhudan1['购车时间']=pd.to_datetime(zhudan1['购车时间'])
    
    c1=zhudan1[zhudan1['购车时间'].isnull()]
    print('还有多少VIN的购车时间是空置:',c1['VIN'].nunique())
    
    c2=zhudan1[zhudan1['修理日期']<zhudan1['购车时间']]
    print('还有多少VIN的购车时间>修理日期:',c2['VIN'].nunique())
    
    print('4.按理所有的购车时间都是小于修理日期的')
    print(len(zhudan1[zhudan1['修理日期']==zhudan1['购车时间']]))
    print(len(zhudan1[zhudan1['修理日期']>zhudan1['购车时间']]))
    print(zhudan1.shape)
    
    print('5.去掉购车即维修的数据')
    zhudan1=zhudan1.loc[zhudan1['修理日期']>zhudan1['购车时间']]
    print(zhudan1.shape)

    del zhudan1['出厂日期']
    del zhudan1['生产年份']
    del zhudan1['生产年份1']
    del zhudan1['date']
    
    for col in ['首保','常规保养','附件安装','一般维修','事故车维修','普通索赔','服务活动',
              '二手车认证维修','内部车辆维修']:
        zhudan1[col]=zhudan1[col].fillna(0)
        
#     print('########加入一级二级车系########') 
#     veh2=veh[['VIN','一级车系','二级车系','车型档次']].drop_duplicates()
#     zhudan1=zhudan1.merge(veh2,on=['VIN'],how='left')
    
    print("########清洗首保数据########")
    zhudan2=clear_shoubao(zhudan1)
    
#     print("#########加入首保经销商,可能涉及多个dealer1,dealer2#########")
#     tmp=zhudan2[zhudan2['首保']==1][['VIN','经销商代码']].rename(columns={'经销商代码':'首保经销商'})
#     print(tmp.shape)
#     print(tmp['VIN'].nunique())
#     zhudan2=zhudan2.merge(tmp,on=['VIN'],how='left')

    print("#########加入维修类型join#########")
    zhudan2=get_one_day_revise_types(zhudan2)
    
    new_buy=zhudan2[['VIN','购车时间']].drop_duplicates().rename(columns={'购车时间':'新购车时间'}).reset_index(drop=True)
    print(new_buy.shape)
    print(new_buy['VIN'].nunique())
    
    del c1
    del c2
    del wrong_purchase_vin
    del zhudan1
    del first
    del veh1
    del df
    del zhudan
    gc.collect()
    
    return zhudan2,changqu,new_buy

In [52]:
zhudan,changqu,new_buy=clear_zhudan(df1,vehicle)

################开始转换成vin+date的宽表################
(5149801, 19)
(5149801, 2)
804533
########加入常去经销商########
(804533, 2)
804533
#############加入第一次进店时间##########
########加入购车时间########
有多少vin出现了购车时间>修理日期: 28623
有多少vin出现了购车那天就被记录了维修的: 16001
有多少记录出现了购车那天就被记录了维修的: 16001
1.先用非空的出厂日期补缺
2.再用大众规则中的生产年份补缺
还有多少VIN的购车时间是空置: 0
还有多少VIN的购车时间>修理日期: 4
3.用修理日期的最早时间补缺
还有多少VIN的购车时间是空置: 0
还有多少VIN的购车时间>修理日期: 0
4.按理所有的购车时间都是小于修理日期的
12862
5136939
(5149801, 25)
5.去掉购车即维修的数据
(5136939, 25)
########清洗首保数据########
####################清洗首保数据#####################
(1).有54个VIN出现首保脏数据
(2).检查脏首保数据是否正确了:
(54, 24)
(54, 2)
54
(3).检查清洗以后的主单数据:
(5136885, 21)
(5136885, 2)
804143
(4).检查首保:
(577953, 21)
(577953, 2)
577953
#########加入维修类型join#########
(804143, 2)
804143


In [53]:
check_df(zhudan)

(5136885, 22)
(5136885, 2)
804143


In [54]:
mis1,mis2=get_miss_cols(zhudan)
print(mis1)
print(mis2.head(50))

[]
Empty DataFrame
Columns: [字段名, 缺失值, 缺失值占比]
Index: []


In [55]:
print("去掉事故车维修和普通索赔后,最后一次进店距离today,超过30*15天的为流失")
print("对于只有事故车或者索赔进店的人,如果购车时间距离today,超过30*15天的为流失")
print('适用于zhudan中所有VIN')
def get_instore_state(df,today):
#     print('此处的df为zhudan所有数据!')
    df.loc[(df['事故车维修']==1)|(df['普通索赔']==1),'流失不计算样本']=1
    
    buyuxia=df.loc[df['流失不计算样本']==1] #取出来的数据,所以可能与另外一部分出现重叠的情况
    yuxia=df.loc[df['流失不计算样本']!=1]
    
    yuxia=yuxia.sort_values(by=['VIN','修理日期'],ascending=False).drop_duplicates(subset=['VIN'],keep='first').reset_index(drop=True)
    yuxia=yuxia[['VIN','修理日期']].drop_duplicates()
    yuxia['修理日期']=pd.to_datetime(yuxia['修理日期'])
    yuxia['距今未进店天数']=(pd.to_datetime(today)-yuxia['修理日期']).apply(lambda x:x.days)
    yuxia.loc[yuxia['距今未进店天数']>=30*15,'在店状态']='已流失'
    yuxia.loc[yuxia['距今未进店天数']<30*15,'在店状态']='未流失'
    
    tmp=yuxia[['VIN','在店状态']].drop_duplicates().reset_index(drop=True)

    other=df.loc[~(df['VIN'].isin(set(yuxia['VIN'])))][['VIN','购车时间']].drop_duplicates()
    other['购车时间']=pd.to_datetime(other['购车时间'])
    other['距今未进店天数']=(pd.to_datetime(today)-other['购车时间']).apply(lambda x:x.days)
    other.loc[other['距今未进店天数']>=30*15,'在店状态']='只有事故索赔进店,已流失'
    other.loc[other['距今未进店天数']<30*15,'在店状态']='只有事故索赔进店,未流失'

    tmp1=other[['VIN','在店状态']].drop_duplicates().reset_index(drop=True)
  
    tmpp=pd.concat([tmp,tmp1],axis=0)

    return tmpp


def get_baoyang_state(df,today):
#     print('此处的df为zhudan中保养数据!')
    df.loc[(df['事故车维修']==1)|(df['普通索赔']==1),'流失不计算样本']=1
    
    buyuxia=df.loc[df['流失不计算样本']==1] #取出来的数据,所以可能与另外一部分出现重叠的情况
    yuxia=df.loc[df['流失不计算样本']!=1]
    
    yuxia=yuxia.sort_values(by=['VIN','修理日期'],ascending=False).drop_duplicates(subset=['VIN'],keep='first').reset_index(drop=True)
    yuxia=yuxia[['VIN','修理日期']].drop_duplicates()
    yuxia['修理日期']=pd.to_datetime(yuxia['修理日期'])
    yuxia['距今未进店天数']=(pd.to_datetime(today)-yuxia['修理日期']).apply(lambda x:x.days)
    yuxia.loc[yuxia['距今未进店天数']>=30*15,'保养状态']='已流失'
    yuxia.loc[yuxia['距今未进店天数']<30*15,'保养状态']='未流失'
    
    tmp=yuxia[['VIN','保养状态']].drop_duplicates().reset_index(drop=True)

    other=df.loc[~(df['VIN'].isin(set(yuxia['VIN'])))][['VIN','购车时间']].drop_duplicates()
    other['保养状态']='仅随事故或索赔才保养'
    tmp1=other[['VIN','保养状态']].drop_duplicates().reset_index(drop=True)

    tmpp=pd.concat([tmp,tmp1],axis=0)

    return tmpp


def get_vin_state(zhudan2,today):
    print('##########在店状态##########')
    instore_df=get_instore_state(zhudan2,today)
    zhudan2=zhudan2.merge(instore_df,on=['VIN'],how='left')
    
    print('##########保养状态##########')
    baoyang_zhudan=zhudan2[(zhudan2['首保']==1)|(zhudan2['常规保养']==1)]
    instore_df1=get_baoyang_state(baoyang_zhudan,today)
    zhudan2=zhudan2.merge(instore_df1,on=['VIN'],how='left')
    zhudan2['保养状态']=zhudan2['保养状态'].fillna('没有保养记录')
    
    print('##########检查二个状态是否有缺失##########')
    print(pd.isnull(zhudan2[['VIN','在店状态','保养状态']]).sum())
    
    zhudan_vin=zhudan2['VIN'].nunique()
    
    print('##########在店状态分布##########')
    tmpp=zhudan2.groupby('在店状态')['VIN'].nunique().reset_index(name='人数')
    tmpp['占比']=tmpp['人数']/zhudan_vin
    print(tmpp.head())
    
    print('##########保养状态分布##########')
    tmpp=zhudan2.groupby('保养状态')['VIN'].nunique().reset_index(name='人数')
    tmpp['占比']=tmpp['人数']/zhudan_vin
    print(tmpp.head())

    return zhudan2

去掉事故车维修和普通索赔后,最后一次进店距离today,超过30*15天的为流失
对于只有事故车或者索赔进店的人,如果购车时间距离today,超过30*15天的为流失
适用于zhudan中所有VIN


In [56]:
zhudan1=get_vin_state(zhudan,today)

##########在店状态##########
##########保养状态##########
##########检查二个状态是否有缺失##########
VIN     0
在店状态    0
保养状态    0
dtype: int64
##########在店状态分布##########
           在店状态      人数        占比
0  只有事故索赔进店,已流失   15857  0.019719
1  只有事故索赔进店,未流失    2548  0.003169
2           已流失  397902  0.494815
3           未流失  387836  0.482297
##########保养状态分布##########
         保养状态      人数        占比
0  仅随事故或索赔才保养    3131  0.003894
1         已流失  420580  0.523016
2         未流失  328276  0.408231
3      没有保养记录   52156  0.064859


In [57]:
def get_last_run(df):
    df['修理日期']=pd.to_datetime(df['修理日期'])
    df['购车时间']=pd.to_datetime(df['购车时间'])
    df=df.sort_values(by=['VIN','修理日期'],ascending=False).reset_index(drop=True)
    df['修理日期_last']=df.groupby('VIN')['修理日期'].shift(-1)
    df['公里数_last']=df.groupby('VIN')['公里数'].shift(-1)
    df.loc[df['修理日期_last'].isnull(),'修理日期_last']=df['购车时间']
    df.loc[df['公里数_last'].isnull(),'公里数_last']=0
    df['天数差']=(df['修理日期']-df['修理日期_last']).apply(lambda x:x.days)
    df['公里数差']=(df['公里数']-df['公里数_last'])
    df['日均差']=df['公里数差']/df['天数差']
    return df

In [58]:
def baoyang_date_clean(baoyang):
    baoyang=get_last_run(baoyang)

    baoyang=baoyang.loc[baoyang['天数差']>7]
    
    tmp=baoyang.groupby('VIN')['修理日期'].nunique().reset_index(name='进店次数')
    baoyang=baoyang.merge(tmp,on=['VIN'],how='left')

    choose1=baoyang.loc[baoyang['进店次数']==1]
    choose11=choose1[choose1['天数差']>7]

    choose2=baoyang.loc[baoyang['进店次数']==2]
    tmp=choose2.groupby('VIN')['天数差'].mean().reset_index(name='保养习惯')
    choose2=choose2.merge(tmp,on=['VIN'],how='left')
    choose2['保养习惯左']=choose2['保养习惯']*0.5
    choose22=choose2.loc[choose2['天数差']>=choose2['保养习惯左']]

    choose3=baoyang.loc[baoyang['进店次数']>=3]
    choose33=choose3[choose3['天数差']>14]

    baoyang=pd.concat([choose11,choose22,choose33],axis=0)
    
    baoyang=get_last_run(baoyang)

    if '历史保养次数' in baoyang.columns.tolist():
        del baoyang['历史保养数据']
    if '保养次数' in baoyang.columns.tolist():
        del baoyang['保养数据']
    tmp=baoyang.groupby('VIN')['修理日期'].nunique().reset_index(name='历史保养次数')
    baoyang=baoyang.merge(tmp,on=['VIN'],how='left')

    return baoyang

In [59]:
def get_baoyang_data(zhudan1):

    baoyang=zhudan1[(zhudan1['首保']==1)|(zhudan1['常规保养']==1)]
    nobaoyang=zhudan1[~(zhudan1['VIN'].isin(set(baoyang['VIN'])))]
    print(baoyang['VIN'].nunique()+nobaoyang['VIN'].nunique()==zhudan1['VIN'].nunique())
    
    print('##############提取baoyang_model###############')

    print('1.保养数据清洗')
    baoyang_model=baoyang_date_clean(baoyang)
    check_df(baoyang_model)
    print('历史保养次数最大值:',baoyang_model['历史保养次数'].max())
    
    print('2.去掉公里数差脏VIN')
    check=baoyang_model[baoyang_model['公里数差']<=0]
    print('其中公里数差有脏数据的VIN有:',check['VIN'].nunique())

    baoyang_model=baoyang_model.loc[~(baoyang_model['VIN'].isin(set(check['VIN'])))]
    cols=['保养习惯', '保养习惯左','进店次数']
    baoyang_model=baoyang_model.drop(cols,axis=1)

    print('3.取出有首保的人群,得到的数据分布是:')
    shoubao=baoyang_model[baoyang_model['首保']==1]
    baoyang_model=baoyang_model[baoyang_model['VIN'].isin(set(shoubao['VIN']))]
    
    print('4.最后检查baoyang_model数据:')
    print('天数差最小值:',baoyang_model['天数差'].min())
    print('公里数差最小值:',baoyang_model['公里数差'].min())
    baoyang_model=baoyang_model.drop(['流失不计算样本','修理日期_last',
                                        '公里数_last', '公里数差','天数差','日均差'],axis=1)
    baoyang_model=baoyang_model.drop_duplicates().reset_index(drop=True)
    check_df(baoyang_model)
    
    print('##############提取baoyang_rule###############')
    baoyang_rule=baoyang[~(baoyang['VIN'].isin(set(baoyang_model['VIN'])))]
    baoyang_rule_df=baoyang_date_clean(baoyang_rule)
    print('天数差最小值:',baoyang_rule_df['天数差'].min())
    print('公里数差最小值:',baoyang_rule_df['公里数差'].min())
    baoyang_rule_df=baoyang_rule_df.drop(['流失不计算样本','修理日期_last',
                                        '公里数_last', '公里数差','天数差','日均差',
                                         '保养习惯', '保养习惯左','进店次数'],axis=1)
    baoyang_rule_df=baoyang_rule_df.drop_duplicates().reset_index(drop=True)
    check_df(baoyang_rule_df)
    print('历史保养次数最大值:',baoyang_rule_df['历史保养次数'].max())
    
    nobaoyang1=baoyang_rule[~(baoyang_rule['VIN'].isin(set(baoyang_rule_df['VIN'])))] #放入规则中的,被清洗掉的人
    nobaoyang1['首保']=0
    nobaoyang1['常规保养']=0
    print(zhudan1['VIN'].nunique()==baoyang_model['VIN'].nunique()+baoyang_rule_df['VIN'].nunique()+nobaoyang['VIN'].nunique()+nobaoyang1['VIN'].nunique())

    print('##############提取nobaoyang###############')
    nobaoyang=pd.concat([nobaoyang,nobaoyang1],axis=0)
    check_df(nobaoyang)
    
    print(zhudan1['VIN'].nunique()==baoyang_model['VIN'].nunique()+baoyang_rule_df['VIN'].nunique()+nobaoyang['VIN'].nunique())
    
    return nobaoyang,baoyang_model,baoyang_rule_df

In [60]:
nobaoyang,baoyang_model,baoyang_rule=get_baoyang_data(zhudan1)

True
##############提取baoyang_model###############
1.保养数据清洗
(3055155, 34)
(3055155, 2)
751441
历史保养次数最大值: 85
2.去掉公里数差脏VIN
其中公里数差有脏数据的VIN有: 10566
3.取出有首保的人群,得到的数据分布是:
4.最后检查baoyang_model数据:
天数差最小值: 8
公里数差最小值: 1.0
(2032856, 25)
(2032856, 2)
549622
##############提取baoyang_rule###############
天数差最小值: 8
公里数差最小值: -494274.0
(1022299, 25)
(1022299, 2)
201819
历史保养次数最大值: 85
True
##############提取nobaoyang###############
(119099, 25)
(119099, 2)
52702
True


In [61]:
baoyang_model_fin=baoyang_model[baoyang_model['修理日期']>=pd.to_datetime('2014-01-01')]
print(baoyang_model_fin['VIN'].nunique())
print(baoyang_model['VIN'].nunique())

baoyang_rule_fin=baoyang_rule[baoyang_rule['修理日期']>=pd.to_datetime('2014-01-01')]
print(baoyang_rule_fin['VIN'].nunique())
print(baoyang_rule['VIN'].nunique())

nobaoyang_fin=nobaoyang[nobaoyang['修理日期']>=pd.to_datetime('2014-01-01')]
print(nobaoyang_fin['VIN'].nunique())
print(nobaoyang['VIN'].nunique())

549622
549622
201819
201819
52702
52702


In [62]:
print('############分别对每个batch数据清洗,然后合并,重新生成新的主单数据,再加入预测人群分类#############')
nobaoyang_fin['人群_一级分类']='没有保养记录'
print(nobaoyang_fin.columns.tolist())
check_df(nobaoyang)

baoyang_model_fin['人群_一级分类']='有保养记录,模型'
print(list(baoyang_model_fin.columns))
check_df(baoyang_model_fin)
print('-'*30)

baoyang_rule_fin['人群_一级分类']='有保养记录,规则'
print(baoyang_rule_fin.columns.tolist())
check_df(baoyang_rule_fin)

############分别对每个batch数据清洗,然后合并,重新生成新的主单数据,再加入预测人群分类#############
['VIN', 'brand', '一般维修', '事故车维修', '二手车认证维修', '修理日期', '修理类型', '公里数', '内部车辆维修', '工时费', '常去经销商', '常规保养', '普通索赔', '服务活动', '经销商代码', '维修金额', '维修金额_折前', '购车时间', '附件安装', '零件费', '首保', '修理类型_new', '流失不计算样本', '在店状态', '保养状态', '人群_一级分类']
(119099, 25)
(119099, 2)
52702
['VIN', 'brand', '一般维修', '事故车维修', '二手车认证维修', '保养状态', '修理日期', '修理类型', '修理类型_new', '公里数', '内部车辆维修', '在店状态', '工时费', '常去经销商', '常规保养', '普通索赔', '服务活动', '经销商代码', '维修金额', '维修金额_折前', '购车时间', '附件安装', '零件费', '首保', '历史保养次数', '人群_一级分类']
(2032856, 26)
(2032856, 2)
549622
------------------------------
['VIN', 'brand', '一般维修', '事故车维修', '二手车认证维修', '保养状态', '修理日期', '修理类型', '修理类型_new', '公里数', '内部车辆维修', '在店状态', '工时费', '常去经销商', '常规保养', '普通索赔', '服务活动', '经销商代码', '维修金额', '维修金额_折前', '购车时间', '附件安装', '零件费', '首保', '历史保养次数', '人群_一级分类']
(1022299, 26)
(1022299, 2)
201819


In [63]:
print('重新合成主单,数据情况如下:')
same_cols=[x for x in [y for y in baoyang_model_fin.columns.tolist() if y in baoyang_rule_fin.columns.tolist()] if x in nobaoyang_fin.columns.tolist()]
print(len(same_cols))
print(same_cols)

fin_zhudan=pd.concat([nobaoyang_fin[same_cols],baoyang_model_fin[same_cols],baoyang_rule_fin[same_cols]],axis=0)
fin_zhudan=fin_zhudan.drop_duplicates().reset_index(drop=True)
check_df(fin_zhudan)

重新合成主单,数据情况如下:
25
['VIN', 'brand', '一般维修', '事故车维修', '二手车认证维修', '保养状态', '修理日期', '修理类型', '修理类型_new', '公里数', '内部车辆维修', '在店状态', '工时费', '常去经销商', '常规保养', '普通索赔', '服务活动', '经销商代码', '维修金额', '维修金额_折前', '购车时间', '附件安装', '零件费', '首保', '人群_一级分类']
(3174254, 25)
(3174254, 2)
804143


In [64]:
def fenlei(dff,today):
    print('分类针对的是zhudan所有人,且这个zhudan是最终被清洗好以后的数据')
    
    dff=dff.sort_values(by=['VIN','修理日期'],ascending=False).reset_index(drop=True)
    dff['修理日期']=pd.to_datetime(dff['修理日期'])
    dff['购车时间']=pd.to_datetime(dff['购车时间'])
    
    dff['车龄']=(pd.to_datetime(today)-dff['购车时间']).apply(lambda x:x.days)
    dff['车龄bin']=dff['车龄'].apply(lambda x:math.ceil(x/180))
    
    if '常规次数' in list(dff.columns):
        del dff['常规次数']
    tmp=dff[dff['首保']!=1]
    tmp=tmp[tmp['常规保养']==1]
    tmpp=tmp.groupby('VIN')['修理日期'].nunique().reset_index(name='常规次数')
    dff=dff.merge(tmpp,on=['VIN'],how='left')
    
    if '首保次数' in list(dff.columns):
        del dff['首保次数']
    tmp=dff[dff['首保']==1]
    tmp['首保次数']=1
    dff=dff.merge(tmp[['VIN','首保次数']].drop_duplicates(),on=['VIN'],how='left')

    for col in ['首保次数','常规次数']:
        dff[col]=dff[col].fillna(0)

    dff.loc[(dff['首保次数']==1)&(dff['常规次数']==0),'人群_二级分类']='有首保,无常规,预测第1次常规'
    dff.loc[(dff['首保次数']==1)&(dff['常规次数']==1),'人群_二级分类']='有首保,1常规,预测第2次常规'
    dff.loc[(dff['首保次数']==1)&(dff['常规次数']==2),'人群_二级分类']='有首保,2次常规,预测第3次常规'
    dff.loc[(dff['首保次数']==1)&(dff['常规次数']>=3),'人群_二级分类']='3次及以上常规,预测接下去常规'
    dff.loc[(dff['首保次数']==0)&(dff['常规次数']==1),'人群_二级分类']='无首保,1次常规,预测第2次常规(购车)'
    dff.loc[(dff['首保次数']==0)&(dff['常规次数']==2),'人群_二级分类']='无首保,2次常规,预测第3次常规'
    dff.loc[(dff['首保次数']==0)&(dff['常规次数']>=3),'人群_二级分类']='3次及以上常规,预测接下去常规'
    dff.loc[(dff['首保次数']==0)&(dff['常规次数']==0)&(dff['车龄bin']<=3),'人群_二级分类']='无保养记录,预测首保'
    dff.loc[(dff['首保次数']==0)&(dff['常规次数']==0)&(dff['车龄bin']>3),'人群_二级分类']='无保养习惯'

    print('查看是否存在没有被分配到的人群')
    c=dff[['VIN','人群_二级分类']].drop_duplicates().reset_index(drop=True)
    print(c.shape)
    print(c['VIN'].nunique())
    print(pd.isnull(c).sum())
    
    print('查看对应的每一类人群的VIN数量')
    tmp=dff.groupby('人群_二级分类')['VIN'].nunique().reset_index(name='人数')
    print(tmp.head(10))
    print(tmp['人数'].sum(0))
    print(dff['VIN'].nunique())
    
    return dff

In [65]:
fin_zhudan_df=fenlei(fin_zhudan,today)

分类针对的是zhudan所有人,且这个zhudan是最终被清洗好以后的数据
查看是否存在没有被分配到的人群
(804143, 2)
804143
VIN        0
人群_二级分类    0
dtype: int64
查看对应的每一类人群的VIN数量
                人群_二级分类      人数
0       3次及以上常规,预测接下去常规  348534
1                 无保养习惯   47094
2            无保养记录,预测首保    5608
3  无首保,1次常规,预测第2次常规(购车)   62805
4      无首保,2次常规,预测第3次常规   10192
5       有首保,1常规,预测第2次常规   79081
6      有首保,2次常规,预测第3次常规   72926
7       有首保,无常规,预测第1次常规  177903
804143
804143


In [66]:
fin_zhudan_df.to_pickle(savep+'final_zhudan.pkl')

In [67]:
baoyang_model_fin=baoyang_model_fin.merge(fin_zhudan_df[['VIN','人群_二级分类']].drop_duplicates(),on=['VIN'],how='left')
baoyang_model_fin.to_pickle(savep+'final_baoyang.pkl')

In [68]:
print("仅为6个固定字段,补充缺失值")
def fill_missing(df,by_col,car_rel):

    if '发动机组号' not in list(df.columns):
        df.rename(columns={'发动机-机组号':'发动机组号'},inplace=True)
    if '变速箱机组号' not in list(df.columns):
        df.rename(columns={'变速箱-机组号':'变速箱机组号'},inplace=True)
        
    del_col=[]
    for col in car_rel:
        tmp=df.groupby([by_col,col])['VIN'].nunique().reset_index(name='人数')
        tmp=tmp.sort_values(by=[by_col,'人数'],ascending=False).drop_duplicates(subset=[by_col],keep='first').reset_index(drop=True)
        del tmp['人数']
        tmp.columns=[by_col,'%s_众数'%col]
        df=df.merge(tmp,on=[by_col],how='left')
        df.loc[df[col].isnull(),col]=df['%s_众数'%col]
        df.loc[df[col]=='nan',col]=df['%s_众数'%col]
        del_col.append('%s_众数'%col)
        gc.collect()
    df1=df.drop(del_col,axis=1)
    return df1

仅为6个固定字段,补充缺失值


In [69]:
def update_vehicle(datap,vehicle,changqu,new_buy,fin_zhudan_df):
    vehicle=vehicle.merge(changqu,on=['VIN'],how='outer')
    vehicle=vehicle.merge(new_buy,on=['VIN'],how='left')
    print('1.检查vin数量')
    print(vehicle.shape)
    print(vehicle['VIN'].nunique())
    
    print('2.去掉又没有进店数据又没有购车时间的')
    vehicle.loc[vehicle['购车时间'].isnull(),'购车时间']=vehicle['出厂日期']
    vehicle.loc[(vehicle['购车时间'].isnull())&(vehicle['第一次修理日期'].notnull()),'购车时间']=vehicle[(vehicle['购车时间'].isnull())&(vehicle['第一次修理日期'].notnull())]['第一次修理日期'].apply(lambda x:pd.to_datetime(x)-relativedelta(days=7))

    no_veh=vehicle[(vehicle['购车时间'].isnull())&(vehicle['常去经销商'].isnull())]
    print(no_veh.shape)
    print(no_veh['VIN'].nunique())
    
    vehicle=vehicle[~(vehicle['VIN'].isin(set(no_veh['VIN'])))]
    
    print('3.检查最终vin人数:')
    print(vehicle.shape)
    print(vehicle['VIN'].nunique())
    
    print('4.加入分类情况:')
    tmp=fin_zhudan_df[['VIN','在店状态','保养状态','人群_一级分类','人群_二级分类']].drop_duplicates()
    print(tmp.shape)
    print(tmp['VIN'].nunique())
    
    vehicle=vehicle.merge(tmp,on=['VIN'],how='left')
    
    for col in ['在店状态','保养状态','人群_一级分类','人群_二级分类']:
        vehicle[col]=vehicle[col].fillna('仅在2014年1月1日前进店过,已流失')
    
    vehicle=get_user(vehicle)
    
    print("##########################开始补充缺失值#########################")
    
    print('1.一级车系和车型档次')
    car_dic1={'一级车系':'途观','车型档次':'中高档'}
    car_dic2={'一级车系':'朗逸','车型档次':'中低档'}
    car_mode=vehicle['一级车系'].mode()[0]
    if car_mode==car_dic1['一级车系']:
        vehicle['一级车系']=vehicle['一级车系'].fillna(car_dic1['一级车系'])
        vehicle['二级车系']=vehicle['二级车系'].fillna('非电动车')
        vehicle['车型档次']=vehicle['车型档次'].fillna(car_dic1['车型档次'])
    elif car_mode==car_dic2['一级车系']:
        vehicle['一级车系']=vehicle['一级车系'].fillna(car_dic2['一级车系'])
        vehicle['二级车系']=vehicle['二级车系'].fillna('非电动车')
        vehicle['车型档次']=vehicle['车型档次'].fillna(car_dic2['车型档次'])
        
    print('2.vin相关属性')
    fill_dic={'性别':'未知','会员卡类型':'NO_CARD','会员卡状态':3,'会员卡年龄':0,'是否绑定微信':0,
              '换购':0,'车辆类型':'私家车','使用性质':'非营运','车主性质':'PERSONAL'}
    for col in fill_dic:
        vehicle[col]=vehicle[col].fillna(fill_dic[col])
    
    print('3.车型代码')
    tmp=vehicle.groupby(['一级车系','车型代码'])['VIN'].nunique().reset_index(name='cnt')
    tmp=tmp.sort_values(by=['一级车系','cnt'],ascending=False).drop_duplicates(['一级车系'],keep='first')
    del tmp['cnt']
    tmp=tmp.rename(columns={'车型代码':'车型代码众数'})
    vehicle=vehicle.merge(tmp,on=['一级车系'],how='left')
    vehicle.loc[vehicle['车型代码'].isnull(),'车型代码']=vehicle['车型代码众数']
    del vehicle['车型代码众数']
    
    print('4.车子相关属性')
    car_rel=['排量', '进气方式', '档位', '技术代码', '发动机组号', '变速箱机组号']
    vehicle=fill_missing(vehicle,'车型代码',car_rel)
    vehicle=fill_missing(vehicle,'一级车系',car_rel)
    
    print('########################匹配常去经销商的省份\城市\城市级别信息###############')
    dealer=pd.read_excel(datap+'VW经销商信息20200513.xls',encoding='gbk')
    cols=['省份', '城市','经销商代码']
    dealer=dealer[cols].drop_duplicates().reset_index(drop=True)
    dealer=dealer[dealer['经销商代码'].notnull()]
    dealer['经销商代码']=dealer['经销商代码'].astype(int).astype(str)
    dealer.rename(columns={'经销商代码':'常去经销商'},inplace=True)
    dealer['城市']=dealer['城市'].apply(lambda x:str(x).replace('市',''))
    dic2={'黔南布依族苗族自治州':'黔南', '恩施土家族苗族自治州':'恩施', '湘西土家族苗族自治州':'湘西',
      '黔西南布依族苗族自治州':'黔西南', '喀什地区':'喀什', '阿克苏地区':'阿克苏',
      '大理白族自治州':'大理', '黔东南苗族侗族自治州':'黔东南', '凉山彝族自治州':'凉山',
      '楚雄彝族自治州':'楚雄', '西双版纳傣族自治州':'西双版纳', '文山壮族苗族自治州':'文山',
      '海西蒙古族藏族自治州':'海西', '巴音郭楞蒙古自治州':'巴音郭楞', '德宏傣族景颇族自治州':'德宏',
      '伊犁哈萨克自治州':'伊犁哈萨克', '昌吉回族自治州':'昌吉', '红河哈尼族彝族自治州':'红河',
      '延边朝鲜族自治州':'延边', '阿拉善盟':'阿拉善', '兴安盟':'大兴安岭'}
    dealer['城市']=dealer['城市'].apply(lambda x:dic2[x] if x in dic2 else x)
    
    vehicle['常去经销商']=vehicle['常去经销商'].fillna(1111111)
    vehicle['常去经销商']=vehicle['常去经销商'].astype(int).astype(str)
    vehicle=vehicle.merge(dealer,on=['常去经销商'],how='left')
    vehicle.loc[vehicle['常去经销商']=='1111111','常去经销商']=np.nan

    city=pd.read_excel(datap+'province_city_info.xlsx',encoding='gbk')
    cols=['city', 'city_class']
    city=city[cols].drop_duplicates().reset_index(drop=True)
    city.columns=['城市','城市级别']
    dic1={'大兴安岭': '大兴安岭', '延边州': '延边', '大理州': '大理', '文山州': '文山', '德宏州': '德宏',
      '楚雄州': '楚雄', '红河州': '红河', '西双版纳州': '西双版纳', '喀什': '喀什', '巴音郭楞州': '巴音郭楞',
      '昌吉州': '昌吉', '阿克苏': '阿克苏', '伊犁哈萨克州': '伊犁哈萨克', '黔西南州': '黔西南',
      '黔东南州': '黔东南', '黔南州': '黔南', '凉山州': '凉山', '恩施州': '恩施', '湘西州': '湘西',
      '海西族州': '海西', '阿拉善': '阿拉善'}
    city['城市']=city['城市'].apply(lambda x:dic1[x] if x in dic1 else x)
#     print(city.shape)
#     print(city['城市'].nunique())
    
    vehicle=vehicle.merge(city,on=['城市'],how='left')

    fill_dic1={'省份':'未知','城市':'未知','城市级别':'未知'}
    for col in fill_dic1:
        vehicle[col]=vehicle[col].fillna(fill_dic1[col])
        
    car_dict={'高档':4, '中高档':3, '中低档':2, '低档':1}
    vehicle['车型档次']=vehicle['车型档次'].map(car_dict)

    city_dict={'三线城市':4, '一线城市':7, '二线发达城市':6, '二线城市':5,
         '四线城市':3, '六线城市':1, '五线城市':2,'未知':0}
    vehicle['城市级别']=vehicle['城市级别'].map(city_dict)
    
    return vehicle

In [70]:
vehicle1=update_vehicle(datap,vehicle,changqu,new_buy,fin_zhudan_df)
mis1,mis2=get_miss_cols(vehicle1)
print('查看缺失的字段:')
print(mis2.head(50))

1.检查vin数量
(850310, 33)
850310
2.去掉又没有进店数据又没有购车时间的
(0, 33)
0
3.检查最终vin人数:
(850310, 33)
850310
4.加入分类情况:
(804143, 5)
804143
##########################开始补充缺失值#########################
1.一级车系和车型档次
2.vin相关属性
3.车型代码
4.车子相关属性
########################匹配常去经销商的省份\城市\城市级别信息###############
查看缺失的字段:
       字段名    缺失值     缺失值占比
0    新购车时间  46167  0.054294
1    常去经销商  45777  0.053836
2  第一次修理日期  45777  0.053836


In [71]:
vehicle1.to_pickle(savep+'vehicle_info.pkl')

### <font color=red>3.得到yanghu_yisun_df

In [72]:
print('###########第一批数据,加载零件信息表#############')
lingjian1=pd.read_pickle(new_datap+'%s_维修零件信息%s.pickle'%(filename,num))
    
cols=['经销商代码', '结算单号', '零件代码', '零件名称', '委托书号', '零件数量']
lingjian1=lingjian1[cols].drop_duplicates().reset_index(drop=True)
lingjian1['经销商代码']=lingjian1['经销商代码'].astype(int).astype(str)

zhudan_pipei1=pd.read_pickle(savep+'zhudan_pipei_lingjian_15selected.pkl')
zhudan_pipei1['经销商代码']=zhudan_pipei1['经销商代码'].astype(int).astype(str)
dt1=lingjian1.merge(zhudan_pipei1,on=['经销商代码','委托书号','结算单号'],how='left')
dt1=dt1[dt1['修理日期'].notnull()]
print(dt1.shape)
print(dt1.columns.tolist())
print(dt1.head())

###########第一批数据,加载零件信息表#############
(22884328, 9)
['经销商代码', '结算单号', '零件代码', '零件名称', '委托书号', '零件数量', 'VIN', '修理日期', '公里数']
      经销商代码          结算单号         零件代码                   零件名称         委托书号  \
0  74301010  BO1401020073   06J115403K      机滤(途观/New Passat)  201401-0090   
1  74301010  BO1401020073   1KD129620B    空气滤芯(途观/New Passat)  201401-0090   
2  74301010  BO1401020073   1KD819653A  空调滤芯 途观/途安/New passat  201401-0090   
3  74301010  BO1401020073  GCB052195Z2                壳牌全合成机油  201401-0090   
4  74301010  BO1401020073  GCN060164Z1               车窗用清洁玻璃水  201401-0090   

   零件数量                VIN       修理日期      公里数  
0   1.0  LSVUD65N0A2870394 2014-01-02  36418.0  
1   1.0  LSVUD65N0A2870394 2014-01-02  36418.0  
2   1.0  LSVUD65N0A2870394 2014-01-02  36418.0  
3   5.0  LSVUD65N0A2870394 2014-01-02  36418.0  
4   2.0  LSVUD65N0A2870394 2014-01-02  36418.0  


In [73]:
dt1['零件代码'] = dt1['零件代码'].apply(lambda x: str(x).replace(' ',''))
dt1['零件代码'] = dt1['零件代码'].apply(lambda x: str(x).replace('#',''))

In [74]:
dt1['修理日期']=pd.to_datetime(dt1['修理日期'])
dt1['零件数量']=dt1['零件数量'].astype(float)
dt1['零件代码'] = dt1['零件代码'].apply(lambda x: str(x).replace(' ',''))
dt1['零件代码'] = dt1['零件代码'].apply(lambda x: str(x).replace('#',''))
dt1['零件名称'] = dt1['零件名称'].apply(str)
dt1=dt1.drop_duplicates().reset_index(drop=True)

In [75]:
tmp=dt1.groupby(['VIN','修理日期','零件代码'])['零件数量'].sum().reset_index()
del dt1['零件数量']
dt1=dt1.merge(tmp,on=['VIN','修理日期','零件代码'],how='left')
dt1=dt1.loc[dt1['零件数量']>0]
dt1.rename(columns = {'零件数量':'配件数量'},inplace = True)
dt1['配件数量'] = dt1['配件数量'].astype('float')
print(dt1.shape)
print(dt1.head())

(22453789, 9)
      经销商代码          结算单号         零件代码                   零件名称         委托书号  \
0  74301010  BO1401020073   06J115403K      机滤(途观/New Passat)  201401-0090   
1  74301010  BO1401020073   1KD129620B    空气滤芯(途观/New Passat)  201401-0090   
2  74301010  BO1401020073   1KD819653A  空调滤芯 途观/途安/New passat  201401-0090   
3  74301010  BO1401020073  GCB052195Z2                壳牌全合成机油  201401-0090   
4  74301010  BO1401020073  GCN060164Z1               车窗用清洁玻璃水  201401-0090   

                 VIN       修理日期      公里数  配件数量  
0  LSVUD65N0A2870394 2014-01-02  36418.0   1.0  
1  LSVUD65N0A2870394 2014-01-02  36418.0   1.0  
2  LSVUD65N0A2870394 2014-01-02  36418.0   1.0  
3  LSVUD65N0A2870394 2014-01-02  36418.0   5.0  
4  LSVUD65N0A2870394 2014-01-02  36418.0   2.0  


In [76]:
pro_code = pd.read_excel(datap+'常用保养养护易损件界定-众调-20200203.xlsx',sheet_name='编号清单-保养养护易损件')
pro_code.loc[pro_code['常用配附件分类'] == '楔形皮带（发电机）','常用配附件分类'] = '楔形皮带'
pro_code.loc[pro_code['常用配附件分类'] == '正时皮带/链条','常用配附件分类'] = '正时皮带'
pro_code.loc[pro_code['常用配附件分类'] == '防冻液/冷却液','常用配附件分类'] = '防冻液'
pro_code.loc[pro_code['常用配附件分类'] == '其他（绿色养护等）','常用配附件分类'] = '绿色养护'

pro_code = pro_code[['常用配附件分类','配附件代码（去空格，去#）']]
pro_code.rename(columns = {'常用配附件分类':'品种','配附件代码（去空格，去#）':'零件号'},inplace = True)
pro_code = pro_code.drop_duplicates()
print(pro_code.head())

    品种         零件号
0  制动液   B000600A3
1  制动液  B000750M1V
2  制动液  B000750M3V
3  制动液  B000750M6V
4  制动液  B000750M7V


In [77]:
dtt = pd.merge(dt1,pro_code, left_on = '零件代码',right_on = '零件号',how = 'left')
dtt = dtt[~pd.isnull(dtt['品种'])]
cols=['VIN', '修理日期', '经销商代码', '公里数', '零件代码', '零件名称', '品种', '配件数量']
dtt=dtt[cols].drop_duplicates().rename(columns={'品种':'find'}).reset_index(drop=True)
print(dtt.shape)

(11261553, 8)


In [78]:
dtt.to_pickle(savep+'yanghu_yisun_df.pkl')

In [79]:
print('run time:',(time.time()-s1)/60)

run time: 36.30233661731084
